# **この Notebook は、[Google Colab: PDF to CSV 変換器を Colab に設置 \[第三話 乱麻篇\] – Categorical data は頼れる味方](https://ggcs.io/2020/08/19/google-colab-pdf-export-03/) の手順詳細版です。**

- Website: ごたごた気流調査所  https://ggcs.io
- GitHub : Gota Gota Current Survey  https://github.com/ggcurrs
- Version 1.0.0
- Date   : 2020-08-18
- Updated: None

In [ ]:
# Script 0

'''以下はすべて Google Colab / Drive が前提となっているので、
local machine で実験する時には適当に path などを変えてください。'''

# Google Drive のマウント。
from google.colab import drive
drive.mount('/content/drive')

# Modules の import.
import os
import sys
# PATH を通す（Python に modules の場所を教える）
# local machine で実験するときは不要。
MODULE_PATH = '/content/drive/My Drive/Colab Notebooks/my-modules'
sys.path.append(MODULE_PATH)
import tabula  # Module の場所を教えたので、import.
import pandas as pd  # 念のため明示的に import しておく。

# ディレクトリ構造を定義する。
# local machine で実験するときは、
# PROJECT_ROOT_PATH = '.' などと適当に変えて使います。
PROJECT_ROOT_PATH = '/content/drive/My Drive/pdf_project'
DATA_PATH   = os.path.join(PROJECT_ROOT_PATH, 'data')

print('準備完了 🍻')

# 今回の作業の流れ
- 前回（[死闘篇](https://ggcs.io/2020/08/11/google-colab-pdf-export-02/)）までの作業で、PDF の粗々の CSV 化まで漕ぎ着けたので、オリジナル PDF の目次に従って、レコード（行 = rows）の並び順をオリジナルに戻すことにします。
- 目標は、州名略記（state_code）> 州ごとの連番（sc_num）の順にソートされた表を作ることです。
- ただし、準備作業は次の順序で行います（Script 番号は Web 版とは異なります）。
0. Setup (Script 0)
1. 作業中のデータの読み込み (Script 1)
2. 州ごとの連番列 (sc_num column) の準備 (Scripts 2-5)
3. 州略記列 (state_code colunn) の準備 (Scripts 6-9)
4. state_code > sc_num の順に並べ替え（Script 10）


# 手順
1. 作業中のデータの読み込み
2. 州ごとの連番列 (sc_num column) の準備
3. 州略記列 (state_code colunn) の準備
4. state_code > sc_num の順に並べ替え（sort）

# 1. 作業中のデータの読み込み
- 前回の作業では、tabula でうまく parse できなかった部分をざっくりと修正し、作業結果を　df_all_intermed.csv として保存したので、これを読み込みます（Script 1）。

In [ ]:
# Script 1
# Load the previously prepared intermediate working file.
df_all_intermed = pd.read_csv(os.path.join(DATA_PATH, 'df_all_intermed.csv'))
df_all_intermed

# 州ごとの連番列 (sc_num column) の準備

## string > float > integer への cast

- CSV ファイルから読み込んだので、sc_num column を含め、data type は全部 string になっています。これだと数値 sort のときに困る（たとえば、string の list ['10', '2', '1'] を sort すると ['1', '10', '2'] になる）ので、***astype()*** method を使って sc_num column は int 型に cast します。
- ところが、'1.0' みたいな string が紛れ込んでいるため、いきなり int にしようとすれば、「1.0 は 整数じゃないぞ💢」（ValueError: invalid literal for int() with base 10: '1.0'）みたいなこと言われるので、*string -> float -> int* と段階を踏んで cast することにします。
- ところが！
- sc_num column を float に cast しようとしたら、'ValueError: could not convert string to float: 'Naga8r5' と言われてしまいました。どうやら、数字と解釈できない文字列 'Naga8r5' が混入していた模様（Script 2）。

In [23]:
# Script 2
df_all_intermed.sc_num.astype(float)

ValueError: ignored

- そこで、 'Naga8r5' を探してその row の様子を見ることにします（Script 3）。確かに、数字ではなくて 'Naga8r5' が入っていますね。
- 早速オリジナル PDF で 'Naga8r5' を検索すると、1 箇所検索に引っかかりました（オリジナル PDF p.37. 'PB|Shahid Bhagat Sing|85'）
- ところが、そこには '85' と書いてあるだけで、'Naga8r5' とは書いてありません？？
- そこで、その PDF 上の '85' をマウス右クリックで copy して text editor に past すると…
- 'Naga8r5' と隠していた正体を顕しました 😲
- どうやら本来の 85 と、どこからか混入した Nagar が絶妙にブレンドされていた模様。
- 原因がわかったので、その場で当該 cell の値を 85 に直して（Script 4）、
- 次に進みます（Script 5）

***Script 2 でいったん実行が止まるので、Runtime menu から Run after してください***

In [24]:
# Script 3
df_all_intermed[df_all_intermed.sc_num == 'Naga8r5']

,state_code,location,sc_num,com_in,com_jp,biz_type
1711,PB,Shahid Bhagat Sing,Naga8r5,SML ISUZU,住友商事<br>いすゞ自動車,"18 輸送機械器具製造業/ Manufacturing – automobile , tw..."


In [25]:
# Script 4
df_all_intermed.at[1711, 'sc_num'] = 85

In [26]:
# Script 5
df_all_intermed.sc_num = df_all_intermed.sc_num.astype(float)
df_all_intermed.sc_num = df_all_intermed.sc_num.astype(int)
# Confirm the results.
df_all_intermed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5102 entries, 0 to 5101
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   state_code  5102 non-null   object
 1   location    5102 non-null   object
 2   sc_num      5102 non-null   int64 
 3   com_in      5102 non-null   object
 4   com_jp      4936 non-null   object
 5   biz_type    5102 non-null   object
dtypes: int64(1), object(5)
memory usage: 239.3+ KB


# 州略記列 (state_code colunn) の準備

## state_code colunn の並び順序（sort order）の定義
- オリジナル PDF の目次（pp.3-4）を基準にするので、オリジナルの目次を tabula で読み込みます（Script 1）。
    - 最初の回（[立志篇](https://ggcs.io/2020/08/05/google-colab-pdf-export-01/)）で pp.5-112 を読み込んだのとまったく同じ手順で、読み込むページだけが異なります。
    - tabula.read_pdf() で返ってくるのは、PDF のページごとに作った DataFrame（以下「DF」）なので、pd.concat() で 1 個の DF に繋げます。

In [ ]:
#Script 6
# Get a list of DFs.
df_toc_list = tabula.read_pdf(
    os.path.join(DATA_PATH, '2018_co_list_jp_r.pdf'), pages='3-4')
# Concatenate DFs in the df_toc_list into one single DF.
df_toc = pd.concat(df_toc_list)
# うまく読み取れたかな？（抜粋）
df_toc[4:43]

- State code の取り出し
  - Column 'Unnamed: 3' に州の略称（state code）が入っているので、これを取り出したい。
  - その際、unique() で重複データ除去（Script 7）。
  - 大丈夫だとは思いますが、念のためオリジナルの PDF を付き合わせて指差確認。
  - 漏れ重複なし。順序にも問題なし！ state_code column 準備完了（Script 8）！

In [ ]:
# Script 7
# Extract state codes from column 'df_toc['Unnamed: 3'].unique()
df_toc['Unnamed: 3'].unique()

In [29]:
# Script 8
# Define state codes and their order.
state_code_order = ['DL', 'HR', 'UP', 'RJ', 'CH', 'PB', 'UK', 'HP', 'JK', 'AS',
       'AR', 'MN', 'ML', 'MZ', 'SK', 'TR', 'WB', 'JH', 'OD', 'BR', 'MH',
       'GJ', 'MP', 'GA', 'CG', 'DD', 'DN', 'KA', 'TN', 'AP', 'TS', 'KL',
       'PY']

## Categorical data を利用した custom sort
- ここでは、Microsoft Excel でいうところの、*並べ替え > ユーザー設定リスト* と同様な操作を行うため、sc_num colunm を Pandas の Categorical date に変換し、上記（Script 9）で定義した カテゴリーの順序も指定します。
- Categorical data のやや詳しい説明は、[Pandas: DataFrame の任意の列を任意の順序でソート する](https://ggcs.io/2020/08/16/pandas-custom-sort/) に書きましたので、ここではちょっと手を抜いて結論だけ（Script 9） 😴
- といいつつちょっと蛇足を加えると、Excel の場合と違って、categorical data は自分自身で自分の順序を覚えているので、sort のときにはいちいち順序を指定する必要はありません。

In [30]:
# Script 9
# 念のため、Copy に対して作業することにします。
df_sorted = df_all_intermed.copy() # sc_num の掃除は既に済んでいる。
# state_code column の data type を categorical にし、順序も定義。
df_sorted.state_code = pd.Categorical(
    df_sorted.state_code, categories=state_code_order, ordered=True)

# Let's SORT!
- 基本的な枠組みは、 普通に sort_values() を使って state_code > sc_num　の階層で sort するだけです（Script 10）。
- 上の準備で苦労した甲斐あって、そのまま普通に sort すれば、state_code は定義された独自の順序で、sc_num は数値（int）順に、それぞれ sort されます。
- だいぶ良い感じになりました 🍻

In [ ]:
# Script 10
# Sort the whole DF.
df_sorted.sort_values(by=['state_code', 'sc_num'], inplace=True)
# reset the index.
df_sorted.reset_index(drop=True, inplace=True)
df_sorted.head(16)

# Sneak Preview

- 既にお気づきのように、この DF にはもう一つ Categorical な column, biz_type があります。
- ちょっと覗いてみると、Script 99 のような状況です。
- [Triad 退治](https://ggcs.io/2020/08/11/google-colab-pdf-export-02/#scr004) の後遺症などもあって、オリジナル PDF のデータがだいぶ撹乱されていますが、次回はその整序などを行いたいと思います。

In [ ]:
# Script 99
sorted(df_sorted.biz_type.unique())